In [42]:
import numpy as np
import pandas as pd
import tensorflow
import torch
from nltk.tokenize import word_tokenize
from nltk import download


#For BARD
from transformers import pipeline

#For mT5-multilingual-XLSum
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


In [56]:
#article 1
ARTICLE1 = '''New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.'''

reference1 = '''Liana Barrientos, 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.'''

In [28]:
def models(Article,summ_txt):
    #Model 1
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    s = summarizer(Article, max_length=130, min_length=30, do_sample=False)
    summ_txt.append(s[0]['summary_text'])
    
    #Model 2
    summarizer = pipeline("summarization", model="philschmid/bart-large-cnn-samsum")
    s = summarizer(Article)
    summ_txt.append(s[0]['summary_text'])
    
    #Model 3
    summarizer = pipeline("summarization", model="Falconsai/text_summarization")
    s = summarizer(Article)
    summ_txt.append(s[0]['summary_text'])
    
    #Model 4
    summarizer = pipeline("summarization", model="knkarthick/MEETING_SUMMARY")
    s = summarizer(Article)
    summ_txt.append(s[0]['summary_text'])
    
    #Model 5
    summarizer = pipeline("summarization", model="lidiya/bart-large-xsum-samsum")
    s = summarizer(Article)
    summ_txt.append(s[0]['summary_text'])
    
    return (summ_txt)


In [29]:
summary = []
models(ARTICLE1,summary)
summary

Token indices sequence length is longer than the specified maximum sequence length for this model (528 > 512). Running this sequence through the model will result in indexing errors


['Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002. If convicted, she faces up to four years in prison.',
 'Liana Barrientos, 39, has been married 10 times between 1999 and 2002 in New York, Westchester County, Long Island, New Jersey or the Bronx. She has pleaded not guilty to two criminal counts of "offering a false instrument for filing in the first degree" on Friday.',
 'Liana Barrientos, now 39, is facing two criminal counts of "offering a false instrument" Prosecutors said the marriages were part of an immigration scam . In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002 .',
 'Liana Barrientos has been married 10 times in 10 years, sometimes only within two weeks of each other. She is accused of faking her marriages as part of an immigration scam.',
 'Liana Barrientos ha

In [41]:
# Download NLTK resources if not already downloaded
download('punkt')

# Tokenize the text into words
reference_tokens = set(word_tokenize(reference1.lower()))
hypotheses_tokens = [set(word_tokenize(hypo.lower())) for hypo in summary]

# Calculate Jaccard similarity
jaccard_similarity = [len(reference_tokens.intersection(tokens)) / len(reference_tokens.union(tokens)) for tokens in hypotheses_tokens]




Jaccard Similarity for Hypothesis 1: 0.4429
Jaccard Similarity for Hypothesis 2: 0.2949
Jaccard Similarity for Hypothesis 3: 0.4000
Jaccard Similarity for Hypothesis 4: 0.2143
Jaccard Similarity for Hypothesis 5: 0.2857


[nltk_data] Downloading package punkt to C:\Users\SUNIL
[nltk_data]     KUMAR\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[0.44285714285714284,
 0.2948717948717949,
 0.4,
 0.21428571428571427,
 0.2857142857142857]

In [44]:
from bert_score import score
P, R, F1 = score(summary, [reference1], lang="en")

# Print BERTScore results
for i, f1 in enumerate(F1):
    print(f"BERTScore for Hypothesis {i + 1}: {f1:.4f}")

# Get the index of the summary with the highest BERTScore
best_summary_index = F1.index(max(F1))
print(f"\nThe best summary is Hypothesis {best_summary_index + 1}")

AssertionError: Different number of candidates and references

In [59]:
from nltk.translate.bleu_score import sentence_bleu

for i, system_summary in enumerate(summary, start=1):
    # Tokenize the summaries
    reference_tokens = [summary.split() for summary in reference1]
    system_tokens = system_summary.split()

    # Calculate BLEU score
    bleu_score = sentence_bleu(reference_tokens, system_tokens)
    print(f"BLEU Score for System Summary {i}: {bleu_score}")

f:\Python\Python39\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
f:\Python\Python39\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
f:\Python\Python39\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


BLEU Score for System Summary 1: 6.957990942521506e-232
BLEU Score for System Summary 2: 6.995501686664742e-232
BLEU Score for System Summary 3: 8.412065649527267e-232
BLEU Score for System Summary 4: 0
BLEU Score for System Summary 5: 7.337741777064293e-232


In [53]:
! pip install meteor

ERROR: Ignored the following versions that require a different python version: 0.1.0 Requires-Python >=3.10,<4.0
ERROR: Could not find a version that satisfies the requirement meteor (from versions: none)
ERROR: No matching distribution found for meteor
